In [ ]:
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Courses/CSE438/Project/Movie Path Link.csv')

In [ ]:
df.shape

(15, 3)

In [ ]:
df.head()

,movie title,year,path
21035,Vampire's Kiss,(1988),/title/tt0098577/
21036,The Gang That Couldn't Shoot Straight,(1971),/title/tt0067124/
21037,The Producers,(2005),/title/tt0395251/
21038,Screwed,(2000),/title/tt0156323/
21039,Home Sweet Home Alone,(2021),/title/tt11012066/


In [ ]:
paths = df['path'].to_numpy()

In [ ]:
print(type(paths))

<class 'numpy.ndarray'>


In [ ]:
movie_title = []
run_time = []
rating = []
total_user_rating = []
genres = []
overview = []
plot_keyword = []
director = []
writer = []
casts = []

In [ ]:
for path in tqdm(paths):
  
  url = f'https://www.imdb.com{path}?ref_=adv_li_tt'
  # print(url)
  webpage = requests.get(url).text
  soup=BeautifulSoup(webpage,'lxml')

  #===================================================================================================================================



  #===================================================================================================================================

  count = 0
  temp_cast = []
  a = soup.find_all('a', href=True, class_="ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link")[:7]
  while count < len(a):
      if count == 0:
          director.append(a[count].text)
      elif count == 1:
          writer.append(a[count].text)
      else: 
          temp_cast.append(a[count].text)

      count+=1
  casts.append(temp_cast)

  #===================================================================================================================================
  movie_title.append(soup.find('h1',class_='sc-b73cd867-0').text)

  try:
      temp = soup.find_all('div',class_='sc-f65f65be-0')
      value = temp[3].find('div',class_='ipc-metadata-list-item__content-container').text
      
  except: run_time.append('not-released')
  else: run_time.append(value)


  try:
      value = soup.find('span',class_='sc-7ab21ed2-1 jGRxWM').text
  except: rating.append('no-rating')
  else: rating.append(value)

      
  try:
      value = soup.find('div',class_='sc-7ab21ed2-3').text
  except: total_user_rating.append(0)
  else: total_user_rating.append(value)

  temp = soup.find_all('li',class_="ipc-inline-list__item ipc-chip__text")
  genre = []
  for i in temp: genre.append(i.text)
  genres.append(genre)


  try:
      value = soup.find('span',class_="sc-16ede01-1").text
  except: overview.append('none')
  else: overview.append(value)




  #===================================================================================================================================

  url2 = f'https://www.imdb.com{path}keywords?ref_=tt_stry_kw'
  webpage2 = requests.get(url2).text
  soup2=BeautifulSoup(webpage2,'lxml')

  temp = soup2.find_all('div',class_="sodatext")[:10]
  plots = []
  for i in temp:
      plots.append(i.text.strip())
  plot_keyword.append(plots)

  #===================================================================================================================================




100%|██████████| 15/15 [00:27<00:00,  1.86s/it]


In [ ]:
print(len(movie_title))
print(len(run_time))
print(len(rating))
print(len(total_user_rating))
print(len(genres))
print(len(overview))
print(len(plot_keyword))
print(len(director))
print(len(casts))
print(len(writer))
print("===================================================================================================================================")

15
15
15
15
15
15
15
15
15
15


In [ ]:
casts = casts[:3984]
len(casts)

15

In [ ]:
df=pd.DataFrame({'movie title': movie_title,
                 'Run Time': run_time,
                 'Rating': rating,
                 'User Rating' : total_user_rating,
                 'Generes' : genres,
                 'Overview': overview,
                 'Plot Kyeword' : plot_keyword,
                 'Director' : director,
                 'Top 5 Casts' : casts,
                 'Writer' : writer}) 
df.shape

(15, 10)

In [ ]:
df.head()

,movie title,Run Time,Rating,User Rating,Generes,Overview,Plot Kyeword,Director,Top 5 Casts,Writer
0,Vampire's Kiss,"$2,000,000 (estimated)",6.0,17K,"[Comedy, Crime, Fantasy]","After an encounter with a neck-biter, a publis...","[psychiatrist, weird behavior, vampire, cockro...",Robert Bierman,"[Nicolas Cage, Maria Conchita Alonso, Jennifer...",Joseph Minion
1,The Gang That Couldn't Shoot Straight,not-released,4.9,1.5K,"[Comedy, Crime]",This is the funny story about two warring Mafi...,"[animated opening credits, gang, police, lion,...",James Goldstone,"[Waldo Salt, Jerry Orbach, Leigh Taylor-Young,...",Jimmy Breslin
2,The Producers,"$45,000,000 (estimated)",6.3,43K,"[Comedy, Crime, Musical]","After putting together another Broadway flop, ...","[scheme, opening night, accountant, swedish, n...",Susan Stroman,"[Thomas Meehan, Nathan Lane, Matthew Broderick...",Mel Brooks
3,Screwed,1 hour 21 minutes,5.6,9.1K,"[Comedy, Crime]",A chauffeur kidnaps his rich boss's dog to hol...,"[slapstick comedy, chauffeur, dog, ransom, car...",Scott Alexander,"[Scott Alexander, Larry Karaszewski, Norm MacD...",Larry Karaszewski
4,Home Sweet Home Alone,not-released,3.6,16K,"[Comedy, Crime, Family]",A married couple tries to steal back a valuabl...,"[christmas, slapstick comedy, toy gun, pool ba...",Dan Mazer,"[Streeter Seidell, John Hughes, Ellie Kemper, ...",Mikey Day


In [ ]:
df.to_csv('IMDB 25k Movie Dataset.csv', index=False)